# Workbook to show how to run RB circuits on IBM Quantum Experience

This is geared towards running 1 and 2 qubit circuits on all the qubits simultaneously as per [Efficient Learning of Quantum Noise](https://arxiv.org/abs/1907.13022).

We use a random X-gate pattern at the beginning to eliminate systematic SPAM bias.

Copyright Robin Harper 2020. See GitHub repo (https://github.com/rharper2/Juqst.jl) for licence.

Note this may well get out of date with qiskit. Unlike the Juqst.jl repo, I am unlikely to keep this up-to date. Qiskit has (historically) changed a lot - but the basic methodology should be instructive.

In [1]:
#Import general libraries (needed for functions)
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

#Import the RB Functions
import qiskit.ignis.verification.randomized_benchmarking as rb

#Import Qiskit classes 
import qiskit
import time
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import __version__

# random is needed for the initial X gates. In order to minimise SPAM (i.e. B=0.5/0.25 in old parlance)
# a random mix of which state we return it to is best. Could seed this if you want.

import numpy.random as rand

# The rest are for saving and logging and list flattening.
import datetime
import os
import pickle
from functools import reduce
from itertools import chain

In [2]:
# Make the data directory if it doesn't exist 
try: 
    os.makedirs("data")
except:
    print()
# Make the temp save directory if it doesn't exist    
try: 
    os.makedirs("temp")
except:
    print()

In [3]:
# This gives us access to our IBMQ account.
# I am assuming you have installed qiskit and have saved your credentials.
# There are plenty of qiskit tutorials out there

from qiskit import IBMQ
IBMQ.load_account() # Load account from disk
IBMQ.providers()    # List all available providers

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]

In [4]:
provider = IBMQ.get_provider(hub='ibm-q')
backends=[i for i in provider.backends()]

# Define the various functions we will need to do the runs and save them

We could move this into a seperate file if needed

In [5]:
# With each sequence run we are going to add some random bitflip gates to get rid of a SPAM parameter
# We need to save this 'mask' with each pickle we get back.

# This just makes it easier to pickle the bit pattern used to randomise the 'expected' measurements
class savedPair:
    def __init__(self, bits,result):
        self.bits = bits
        self.result = result


In [6]:
from qiskit.providers import JobStatus


def savePairs(bs,js,start,savePrefix,forced = False):
    """
    savePairs(bs,js,start,savePrefix,forced = False)
    
    Designed to save jobs as and when the results become available.
    We don't want to block, however, rather we will iterate through a list
    and try to deal gracefully with jobs that are not ready or that
    give us an error when we try and retrieve them.
    If the file is there, the job will be skipped UNLESS forced is true.
    As long as we have the job_id (which should have been saved seperately)
    we should be able to retrieve them later anyway.
    bs: the bitstrings associated with the jobs
    js: a list of the jobs we have
    start: the number the jobs start at
    savedPrefix: What do we want the saved pickles to start with.
    forced: Set it to true if you want to save the job, even if we think it has been saved before.
    """
    for (idx,j) in enumerate(js):
        saved_this_one = False
        bits = bs[idx]
        if forced or not os.path.isfile(savePrefix+str(start+idx)+'.pickle'):
           try:
              if j.status() == JobStatus.DONE:
                 res = j.result()
                 with open(savePrefix+str(start+idx)+'.pickle','wb') as f:
                        pickle.dump(savedPair(bits,res),f)
                 saved_this_one = True
           except ApiError as ex:
              print("There was a api exception error")
              template = "An exception of type {0}. Args \n{1!r}"
              message = template.format(type(ex).__name__,ex.args)
              print(message)
              print("Ignoring this for now, but NOT saved.")
           except Exception as ex2:
              print("There was a non-api exception error")
              template = "An exception of type {0}. Args \n{1!r}"
              message = template.format(type(ex2).__name__,ex2.args)
              print(message)
              print("Ignoring this for now, but NOT saved.")
           if saved_this_one == True:
              now = datetime.datetime.now()
              print("Saved: "+str(idx+start)+": " + now.strftime("%d %B: %r "))

In [7]:
def stringifyBits(bits):
    """ Takes an array of bits (i.e. 1 or 0) and turns them into a string """
    return [''.join([str(x) for x in i[::-1]]) for i in bits]

def mashable(s1,s2):
    """ XOR on strings """
    s3 = ''
    for i in range(len(s1)):
        if s1[i] == s2[i]:
            s3 = s3 + '0'
        else:
            s3 = s3 + '1'
    return s3

In [8]:
# If you don't specify an initial layout MAKE SURE the qiskit transpiler doesn't mix and match the qubit location.
def generateInitialLayout(q):
  """ Pass in the qregister and generate a one to one initial layout """
  initial_layout = {}
  for (idx,q) in enumerate(q):
    initial_layout[q] = idx
  return initial_layout

In [9]:
def createARun(qubits,lengths,batching,sets): 
    """
      createARun(qubits,lengths,batching,sets)
      
      Generates RB runs
      qubits: number of qubits
      lengths: list of lengths for the runs
      batching: How many complete seqeunces of runs (total submissions will = len(lengths)*batching)
      sets: how you want the qubits done e.g. [[0,1],[2],[3]] will be a 4 qubit, 2 qubit RB on 0,1 and single qubit on 2 and 3
      
      Returns: a tuple(bits,circuits,layout)
               bits = patterns of x gates used. Note this is in [Q0,Q1,Q2...] order
               circuits = the circuits to run - as a single array not nested
               initial_layout - the layout to be passed to the execute functions.
      """
    # Generate RB circuits 
    # number of qubits
    nQ=qubits
    rb_opts = {}
    # Number of Cliffords in the sequence
    rb_opts['length_vector'] = lengths
    # Number of random sequences to batch up
    rb_opts['nseeds'] = batching
    #Default pattern
    rb_opts['rb_pattern'] = sets

    rb_circs, xdata = rb.randomized_benchmarking_seq(**rb_opts)    

    initial_layout=generateInitialLayout(rb_circs[0][0].qregs[0])
    
    # Generate random bit pattern to apply X - gates.
    # We need one for each seqeunce. and each batch
    
    bits= [[rand.randint(0,2,nQ) for _ in range(len(lengths))] for _ in range(batching)]
    
    
    # Use the bit pattern in bits to apply X gates before the circuits generated.
    for batch in range(batching):
        for (ib,b) in enumerate(bits[batch]):
            qregs = rb_circs[batch][ib].qregs
            cregs = rb_circs[batch][ib].cregs
            # Create a temporary circuit sharing the qubits and creg with our RB circuit
            p=qiskit.QuantumCircuit(*qregs,*cregs)
            # Step through the random array and if we have a 1 put an X circuit on the qubit of that circuit.
            for (q,doX) in enumerate(b):
                if doX ==1:
                    p.x(q)
                    # print(ib,b,q,"setting to x")
            # Go throught the gates on the temporary circuit and add them to the beginning of our RB circuit.
            for t in p.data:
                rb_circs[batch][ib].data.insert(0,t)
    flatCircuits = list(chain(*rb_circs))
    return (bits,flatCircuits,initial_layout)
   


In [10]:
def send(circuits,initial_layout,backend,shots=1024):
    """
      backend: The backend you want
      shots: the number of shots to take.
      
      returns: job created and sent.
    """
    # This will not be idiomatic Python - should find out `better` way to write this
    # Basically if we don't get an error free response from a backend query, sleep and try again.
    max_credits=5
    errorFree = False
    coupling_map = backendToUse.configuration().coupling_map
    # If its down wait for it to come back up. 
    # This attempts to stop everything going horribly wrong cause the machine is down for e.g. maintenance.
    while not errorFree:
        try:
            while not backend.status().operational:
                now = datetime.datetime.now()
                print("Device not operational at", now.strftime("%d %B: %r"), ", sleeping for a bit")
                time.sleep(60*10) # 10 minutes.
            errorFree = True
        except KeyboardInterrupt:
            print("Keboard interrupt whilst waiting for the backend to come back up")
            print("Going to return with what has been done so far")
            return (jobsList,bitFlips)
        except Exception as ex:
            print("Got an exception whilst checking the backend operational status. It is almost certainly just a maintenance issue.")
            template = "An exception of type {0}. Args \n{1!r}"
            message = template.format(type(ex).__name__,ex.args)
            print(message)
            print("Ill sleep for 30 minutes and then recheck.")
            time.sleep(60*30)
    # if we run with simulator take out the coupling.
    errorFree = False
    while not errorFree:
        try:
            job_exp = execute(circuits, backend=backend, shots=shots, max_credits=max_credits,initial_layout=initial_layout,coupling_map=coupling_map)
            errorFree=True
        except Exception as ex:
            print("Got an exception whilst trying to submit the job. Probably a too many concurrent etc.")
            template = "An exception of type {0}. Args \n{1!r}"
            message = template.format(type(ex).__name__,ex.args)
            print(message)
            print("Ill sleep for 3 minutes and then recheck.")
            time.sleep(60*3)
    return job_exp
    


In [11]:
def getAndSaveJobID(bits,job_exp,index,id_save_name="temp/JobIDandBit_temp_"):
    """
        Given a list of bits (x gates applied) and a list of jobs (returned from execute)
        We retrieve the job_id and then save it with the corresponding bitpattern of x gates.
        in a pickle starting at index and using a prefix
        of id_save_name (default JobIDandBit_temp_)
        
        The purpose of this is incase something goes wrong before the
        job itself can be saved - if we have the bits and the id we can
        always get the job results later.
    """
    # So now we want the jobid so we can pickle it. Can't pickle the job_exp as its threadlocked.
    # We can get errors trying to get the jobid from the submitted job
    # Try to deal with them gracefully.
    try:
        jobId = job_exp.job_id()
        with open(id_save_name+str(index)+".pickle",'wb') as f:
            pickle.dump(savedPair(bits,jobId),f)
        now = datetime.datetime.now()
        print("Saved bits and jobid: ",index,":", now.strftime("%d %B: %r"))
    except Exception as ex:
        template = "An exception of type {0}. Args \n{1!r}"
        message = template.format(type(ex).__name__,ex.args)
        print(message)
        print("We got an error, probably timing, things might be busy job ids not available. etc")
        time.sleep(60*2)
        try:
            jobId = job_exp.job_id()
            with open(id_save_name+str(index)+".pickle",'wb') as f:
                pickle.dump(savedPair(bits,jobId),f)
            now = datetime.datetime.now()
            print("Saved bits and jobid: ",index,": ", now.strftime("%d %B: %r"))
        except Exception as ex2:
            print("That really didn't work, so just NOT SAVING, well only the bits. You will have the job in the returned lists, lose that an you have to re-run this.")
            template = "An exception of type {0}. Args \n{1!r}"
            message = template.format(type(ex2).__name__,ex2.args)
            print(message)
            # Its a bit pointless just saving the bits, but I suppose we can query the backend so here we go.
            with open(id_save_name+str(index)+".pickle",'wb') as f:
                pickle.dump(savedPair(bits,[]),f)
    return jobId



In [12]:
def countQueuedJobs(jobs):
    """
    Checks how many jobs are queued, running or about to be queued
    The 'fair use' policy appears to only allow you to stack the queue to a certain amount
    I think different 'rights' allow different numbers to be queued..
    """
    count = 0
    for j in jobs:
        job_status = j.status()
        if job_status == JobStatus.QUEUED or \
           job_status == JobStatus.INITIALIZING or \
           job_status == JobStatus.VALIDATING or \
           job_status == JobStatus.RUNNING:
            count += 1
    return count

In [13]:
def sendAJobWhenOk(nQ,lengths,batchSize,sets,backendToUse,start,end,maxQueue = 3,shots=1024,saveFile="date/Melbourne15_"):
    """
    sendAJobWhenOk(nQ,lengths,batchSize,sets,backendToUse,start,end,maxQueue = 3,saveFile="Melbourne15")
    Sends off a number of jobs (from start to end)
    Only allows queue size to be 3 (you may want to alter if you can have more in queue)
    Saves numbered with saveFile as a prefix.
    You also need to supply:
      nQ, the number of qubits
      lengths, the sequence lenths as a list
      batchSize, the number of experiments in a single batch - qasm limits will limit this depending on lengths
      backendToUse, err the backend you want to use
      start, start numbering here
      end, end numbering here (well one before here I suppose).
      shots: (default = 1024) number of shots per circuit
    """
    all_jobs = []
    all_bits = []
    all_job_ids = []
    for toDo in range(start,end):
        (bits,circuit,layout) = createARun(nQ,lengths,batchSize,sets) 
        good_to_go = False
        while not good_to_go:
            counted = countQueuedJobs(all_jobs)
            if counted >= maxQueue:
                print("We have {} jobs in queue, sleeping: {}".format(counted,datetime.datetime.now()))
                time.sleep(60*2)
                savePairs(all_bits,all_jobs,1,saveFile,forced = False)
            else:
                good_to_go = True
        job_exp = send(circuit,layout,backendToUse,shots=1024)
        job_id = getAndSaveJobID(bits,job_exp,toDo)
        all_jobs.append(job_exp)
        all_job_ids.append(job_id)
        all_bits.append(bits)
    counted = countQueuedJobs(all_jobs)
    while counted > 0:
        print("Still have {} jobs running: {}. Waiting for 120 seconds.".format(counted,datetime.datetime.now()))
        time.sleep(2*60)
        counted = countQueuedJobs(all_jobs)
    savePairs(all_bits,all_jobs,1,saveFile,forced = False)
    return (all_bits,all_jobs,all_job_ids)

# Initial runs - here we test everything with the online simulator

<h2 style="color:red">Note:</h2>

Choosing the lengths sensibly is very important for reliable results. With normal RB you have a reasonable idea of the fidelity and you can choose the lengths so the first is near the beginning (but with a high probability of success) and the second is close to the 'end' of the decay curve but **not** after we are at the maximally mixed state. ([Statistical Analysis of Randomized Benchmarking](https://arxiv.org/abs/1901.00535)) gives some guidance as to how to choose this.

Here we need statistics that will allow us to fit lots of curves, with varying fidelity decays, so we need more data points, and then we can use some sensible fitting rules to work out which tail ones to through away. This is all detailed, but essentially we discount any length where the value is less than $17/64^{\text{ths}}$ of the initial value (assuming a decay to 0, if you have a constant factor then that will need to be taken into account). Some of the eigenvalues in these near term devices will be lowish so we need a good concentration of shortish lengths. Some will be very high (the single qubit fidelities) and so we need a reasonable tail.

If we use the two qubit protocol things are quite different. The good fidelities are poor and the bad fidelities are pluto. In our analysis I ended up using a single qubit/idle initial gate for the first sequence (even though this has some technical SPAM related problems) in order to get the really bad fits anchored in the right place - and we need a large number of small sequences because of the extreme decays. This is, again, detailed in the relevant workbooks in the GitHub repo (https://github.com/rharper2/Juqst.jl) for the Julia analysis code.

In [14]:
# Choose the lengths we want:
lengths = [1,5,10,15,20,30,45,60,75,90,110]
# Choose the layout we want (here 15 qubits, all single qubit)
nQ = 15
sets = [[0],[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12],[13],[14]]


# Choose how many to batch - this will depend on maximum qasm size, which will in turn depend
# on the number of qubits, the lengths chosen and whether they are 1 or two qubits.
# To high and you will get an error when you try to submit
# Higher is more sequences per call.
batchSize = 3




In [15]:
backendToUse = backends[0]
backendToUse.name()

'ibmq_qasm_simulator'

In [16]:
(all_b,all_j,all_j_id) = sendAJobWhenOk(15,lengths,3,sets,backendToUse,1,6,saveFile="data/Simulator")

Saved bits and jobid:  1 : 17 June: 09:49:18 AM
Saved bits and jobid:  2 : 17 June: 09:49:48 AM
Saved bits and jobid:  3 : 17 June: 09:50:20 AM
Saved bits and jobid:  4 : 17 June: 09:51:03 AM
Saved bits and jobid:  5 : 17 June: 09:51:35 AM
Still have 1 jobs running: 2020-06-17 09:51:37.022011. Waiting for 120 seconds.


## Check we got the results we expected:
To check its right, we should check that when we XOR with the X-gate pattern we get all the shots. 

Because the simulator has no noise, we expect the pattern 000...000 to be 1024

In [17]:
# Flatten out the Bits, so we can iterate through a list
flatBits = list(chain(*all_b))
flatBits = list(chain(*flatBits))
fbs = stringifyBits(flatBits)

In [18]:
results = [j.result() for j in all_j]

In [19]:
counts = [r.get_counts() for r in results]

In [20]:
flatCounts = list(chain(*counts))


In [22]:
for (ix,c) in enumerate(flatCounts):
    for k in c.keys():
        print("{}: {}".format(mashable(fbs[ix],k),c[k]))

000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
000000000000000: 1024
0000000000

# Run on an actual Device

This is always a bit more fraught, because the queues are larger, the device goes offline and sometimes (certainly in the past) jobs go missing etc - but fingers crossed!

In [23]:
backends

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [24]:
backendToUse = backends[2]

In [ ]:
# You might want to do, say, 250*4 = 1000 sequence run.
# The time stamps will show how long it took.
# The text below is just an example of the output.
(all_b,all_j,all_j_id) = sendAJobWhenOk(15,lengths,4,sets,backendToUse,0,250,saveFile="data/Melbourne15_17June2020_")

Saved bits and jobid:  0 : 17 June: 08:41:27 PM
Saved bits and jobid:  1 : 17 June: 08:42:02 PM
Saved bits and jobid:  2 : 17 June: 08:42:38 PM
We have 3 jobs in queue, sleeping: 2020-06-17 20:43:43.826976
We have 3 jobs in queue, sleeping: 2020-06-17 20:45:48.050305
We have 3 jobs in queue, sleeping: 2020-06-17 20:47:52.701443
Saved: 1: 17 June: 08:50:03 PM 
Saved: 2: 17 June: 08:50:07 PM 
Saved bits and jobid:  3 : 17 June: 08:50:36 PM
Saved bits and jobid:  4 : 17 June: 08:52:08 PM
Saved bits and jobid:  5 : 17 June: 08:52:52 PM
We have 3 jobs in queue, sleeping: 2020-06-17 20:53:13.150042
Saved: 3: 17 June: 08:55:19 PM 
We have 3 jobs in queue, sleeping: 2020-06-17 20:55:22.776772
We have 3 jobs in queue, sleeping: 2020-06-17 20:57:26.790984
Saved: 4: 17 June: 08:59:33 PM 
Saved bits and jobid:  6 : 17 June: 09:05:34 PM
Saved bits and jobid:  7 : 17 June: 09:06:22 PM
Saved bits and jobid:  8 : 17 June: 09:08:18 PM
We have 3 jobs in queue, sleeping: 2020-06-17 21:08:44.669956
Saved:

We have 3 jobs in queue, sleeping: 2020-06-18 00:02:22.030079
Saved: 59: 18 June: 12:04:26 AM 
Saved: 60: 18 June: 12:04:29 AM 
Saved: 61: 18 June: 12:04:34 AM 
Saved bits and jobid:  63 : 18 June: 12:05:51 AM
We have 3 jobs in queue, sleeping: 2020-06-18 00:06:03.185327
We have 3 jobs in queue, sleeping: 2020-06-18 00:08:07.673593
Saved: 62: 18 June: 12:10:11 AM 
Saved bits and jobid:  64 : 18 June: 12:10:32 AM
We have 3 jobs in queue, sleeping: 2020-06-18 00:10:43.617124
Saved: 63: 18 June: 12:12:48 AM 
Saved: 64: 18 June: 12:12:52 AM 
Saved bits and jobid:  65 : 18 June: 12:15:56 AM
Saved bits and jobid:  66 : 18 June: 12:16:26 AM


In [27]:
now = datetime.datetime.now()
print("Printed at: " + now.strftime("%d %B: %r "))
qiskit.tools.backend_monitor(backendToUse)

Printed at: 17 June: 07:28:34 AM 
ibmq_16_melbourne
Configuration
-------------
    n_qubits: 15
    operational: True
    status_msg: active
    pending_jobs: 48
    backend_version: 2.1.0
    basis_gates: ['id', 'u1', 'u2', 'u3', 'cx']
    local: False
    simulator: False
    conditional: False
    quantum_volume: None
    online_date: 2018-11-06 05:00:00+00:00
    url: None
    meas_map: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]]
    backend_name: ibmq_16_melbourne
    description: 15 qubit device
    n_registers: 1
    credits_required: True
    allow_object_storage: True
    open_pulse: False
    sample_name: albatross
    coupling_map: [[0, 1], [0, 14], [1, 0], [1, 2], [1, 13], [2, 1], [2, 3], [2, 12], [3, 2], [3, 4], [3, 11], [4, 3], [4, 5], [4, 10], [5, 4], [5, 6], [5, 9], [6, 5], [6, 8], [7, 8], [8, 6], [8, 7], [8, 9], [9, 5], [9, 8], [9, 10], [10, 4], [10, 9], [10, 11], [11, 3], [11, 10], [11, 12], [12, 2], [12, 11], [12, 13], [13, 1], [13, 12], [13, 14], [14, 0], 

# That's it.

After this has run, I believe everything is saved in the pickles. The other workbook shows how to extract the data from the pickles that we need for the Efficient Learning algorithm. You could of course use the list of jobs you have here etc, up to you!